# 초기 설정

## 수집 앱 URL

In [ ]:
URL = "https://newneek.co/"

# 패키지 import

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
import pandas as pd

In [ ]:
# anaconda python으로 interpreter 설정하기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager


# 더보기 클릭

In [ ]:
def more(modal):
    

In [ ]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
                driver.manage().timeouts().implicitlyWait(2000, TimeUnit.MILLISECONDS)
                driver.manage().timeouts().pageLoadTimeout(40, TimeUnit.SECONDS)
                driver.manage().timeouts().setScriptTimeout(60, TimeUnit.SECONDS)
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

# 기사 클릭해서 page source 가져오기

# 데이터 크롤링

In [ ]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(URL)
wait = WebDriverWait(driver, 5)

# 페이지 당 12개 리뷰 불러오기 (class가 card인 것 가져오기)

# 페이지 렌더링 대기
page_xpath = '/html/body/div/section'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, page_xpath)))

card_news = '/html/body/div/section/div/a[1]'
card_news_button = driver.find_element(By.XPATH, card_news)
card_news_button.click()
time.sleep(1) 
driver.save_screenshot('AFTER.png')    
# # 순서대로 들어가서 page source 가져오고 나오기

# # 더보기 누르기
# button_xpath = '/html/body/div/section/nav/button'
# button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
# # '더보기' 버튼 클릭
# driver.find_element(By.XPATH,button_xpath).click()


In [ ]:
driver = webdriver.Chrome()
driver.get(URL)
wait = WebDriverWait(driver, 5)

while 1: 
    # 페이지 당 12개 리뷰 불러오기 (class가 card인 것 가져오기)
    card_news = driver.find_element(By.CLASS_NAME, "card")
    print(card_news)
    # 순서대로 들어가서 page source 가져오고 나오기

    # 더보기 누르기

In [ ]:
driver = webdriver.Chrome()
# 페이지 열기
driver.get(URL)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)

# '리뷰 모두 보기' 버튼 렌더링 확인(path 수정 @2022-06-22)
all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
# '리뷰 모두 보기' 버튼 클릭
driver.find_element(By.XPATH,all_review_button_xpath).click()

# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

# 더보기 함수
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

# HTML Parsing

In [ ]:
# html parsing하기
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

# 크롬 드라이버 종료

In [ ]:
driver.quit()

# HTML 데이터 저장

In [ ]:
# html 데이터 저장
with open("../dataset/korea_3.html", "w", encoding = 'utf-8') as file: #이름 바꾸기
    file.write(str(soup_source))

## 데이터프레임 변환

In [ ]:
# 리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 리뷰 데이터 저장용 배열
dataset = []
# 데이터 넘버링을 위한 변수
review_num = 0 
# 리뷰 1개씩 접근해 정보 추출
for review in review_source:
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
    
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    try:
        content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출
    except:
        content = ''
    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

## 데이터프레임 저장

In [ ]:
df = pd.DataFrame(dataset)
df.to_csv('../dataset/korea_dataset.csv', encoding = 'utf-8-sig') # csv 파일로 저장/ 이름 바꾸기

# 리뷰 데이터 불러오기

In [ ]:
# 저장한 리뷰 정보 불러오기
df = pd.read_csv('../dataset/korea_dataset.csv', encoding = 'utf-8-sig') # 이름 바꾸기
df = df.drop(['Unnamed: 0', 'id', 'date', 'dateMonth', 'dateDay', 'userName'], axis = 1) # 불필요한 칼럼 삭제
df.to_csv('../dataset/korea_review.csv', encoding = 'utf-8-sig') # csv 파일로 저장/ 이름 바꾸기